In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re,glob
AUTO = tf.data.experimental.AUTOTUNE
DIM = 600 
IMAGE_SIZE=[DIM,DIM]

BATCH_SIZE = 16  
DATA_PATH = "../input/ranzcr-clip-catheter-line-classification/"
OUTPUT_PATH = "./"

import sys

!pip install ../input/kerasapplications/keras-applications-master/ 
package_path = '../input/effici/efficientnet-master/'
sys.path.append(package_path)

#test
from efficientnet import tfkeras as efn

TEST_FILENAMES = tf.io.gfile.glob('../input/ranzcr-clip-catheter-line-classification/test_tfrecords/*.tfrec') # predictions on this dataset should be submitted for the competition
print(TEST_FILENAMES) 

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.image.resize(image, [DIM, DIM])
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_unlabeled_tfrecord(example):

    UNLABELED_TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'StudyInstanceUID': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['StudyInstanceUID']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled = True, ordered = False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # use data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_unlabeled_tfrecord, num_parallel_calls = AUTO) # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset


def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)

In [ ]:
models1=[]  

#for filename in glob.glob('../input/eff7trained/*s.h5'): #assuming gif  
model = tf.keras.models.load_model('../input/efficetmodels/model_1s.h5/model_eff7-fold_1-best.h5', custom_objects = None )  
models1.append(model)
model = tf.keras.models.load_model('../input/efficetmodels/model_2s.h5/model_eff7-fold_2-best.h5', custom_objects = None )  
models1.append(model)
model = tf.keras.models.load_model('../input/efficetmodels/model_3s.h5/model_eff7-fold_3-best.h5', custom_objects = None )  
models1.append(model)
model = tf.keras.models.load_model('../input/efficetmodels/model_4s.h5/model_eff7-fold_4-best.h5', custom_objects = None )  
models1.append(model)


In [ ]:
test_ds = get_test_dataset(ordered=True) 
test_images_ds = test_ds.map(lambda image, idnum: image)

labels = ['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']
mean =(models1[0].predict(test_images_ds) 
       +models1[1].predict(test_images_ds) 
       +models1[2].predict(test_images_ds) 
       +models1[3].predict(test_images_ds))/4.0

In [ ]:
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') 

submission = pd.DataFrame(mean, columns = labels)

submission.insert(0, "StudyInstanceUID", test_ids, False) 
submission['StudyInstanceUID'] = submission['StudyInstanceUID'].apply(lambda x: x.rstrip(".jpg"))
submission.to_csv('submission.csv', index=False)

print("Done")